In [1]:
library(dplyr)
library(tidyr)
library(readr)
library(doMC)
library(stringr)
registerDoMC(cores = 28)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: foreach

Loading required package: iterators

Loading required package: parallel



In [2]:
flist <- Sys.glob('/pool001/mfzhao/mobility/*/*')
eu_mobility <- foreach(f = flist[!str_detect(flist, 'united_states')], .combine = 'rbind') %dopar% read_csv(f)
us_mobility <- foreach(f = flist[str_detect(flist, 'united_states')], .combine = 'rbind') %dopar% read_csv(f)

In [8]:
us_mobility %>%
    arrange(ds, polygon_id) %>%
    filter(ds == as.Date('2020-03-01'))

ds,crisis_name,polygon_id,polygon_name,age_bracket,gender,baseline_name,baseline_type,all_day_bing_tiles_visited_relative_change,all_day_ratio_single_tile_users,external_polygon_id_type,external_polygon_id
<date>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<chr>
2020-03-01,US_county,1001,Autauga,Overall,Overall,full_february_except_presidents_day,DAY_OF_WEEK,0.0484494125,0.1891036,FIPS,01001
2020-03-01,US_county,1003,Baldwin,Overall,Overall,full_february_except_presidents_day,DAY_OF_WEEK,0.1429764399,0.1720912,FIPS,01003
2020-03-01,US_county,1005,Barbour,Overall,Overall,full_february_except_presidents_day,DAY_OF_WEEK,0.1948150081,0.1477137,FIPS,01005
2020-03-01,US_county,1007,Bibb,Overall,Overall,full_february_except_presidents_day,DAY_OF_WEEK,0.0784205232,0.1651653,FIPS,01007
2020-03-01,US_county,1009,Blount,Overall,Overall,full_february_except_presidents_day,DAY_OF_WEEK,0.0364588878,0.1999208,FIPS,01009
2020-03-01,US_county,1011,Bullock,Overall,Overall,full_february_except_presidents_day,DAY_OF_WEEK,0.1600880985,0.1542295,FIPS,01011
2020-03-01,US_county,1013,Butler,Overall,Overall,full_february_except_presidents_day,DAY_OF_WEEK,0.1525933141,0.1759363,FIPS,01013
2020-03-01,US_county,1015,Calhoun,Overall,Overall,full_february_except_presidents_day,DAY_OF_WEEK,0.0431625529,0.1777367,FIPS,01015
2020-03-01,US_county,1017,Chambers,Overall,Overall,full_february_except_presidents_day,DAY_OF_WEEK,0.0683377245,0.1773198,FIPS,01017


In [3]:
eu_mobility %>% 
    filter(baseline_name != 'limited_february') %>%
    select(key = external_polygon_id,
           ds, 
           btvrc = all_day_bing_tiles_visited_relative_change, 
           rstu = all_day_ratio_single_tile_users) %>%
    arrange(key, ds) %>%
    mutate(key = ifelse(key == 'UKN0A', 'UKN10', key),
           key = ifelse(key == 'UKN0B', 'UKN11', key),
           key = ifelse(key == 'UKN0C', 'UKN12', key),
           key = ifelse(key == 'UKN0D', 'UKN13', key),
           key = ifelse(key == 'UKN0E', 'UKN14', key),
           key = ifelse(key == 'UKN0F', 'UKN15', key),
           key = ifelse(key == 'UKN0G', 'UKN16', key)) -> eu_mobility

us_mobility %>% 
    filter(baseline_name != 'limited_february') %>%
    select(key = external_polygon_id,
           ds, 
           btvrc = all_day_bing_tiles_visited_relative_change, 
           rstu = all_day_ratio_single_tile_users) %>%
    mutate(key = str_pad(key, 5, pad = '0')) %>%
    arrange(key, ds) -> us_mobility

In [4]:
eu_pop <- read_delim('/pool001/mfzhao/geo_data/europe_population.tsv', '\t', escape_double = FALSE, trim_ws = TRUE)

eu_pop %>%
    separate('sex,unit,age,geo\\time', into = c('sex', 'unit', 'age', 'key'), sep = ',') %>%
    filter(sex == 'T', age == 'TOTAL') %>%
    select(key, n = `2018`) %>%
    mutate(n = str_replace(n, ' p', ''),
           n = as.numeric(n)) -> eu_pop

Parsed with column specification:
cols(
  `sex,unit,age,geo\time` = col_character(),
  `2019` = col_character(),
  `2018` = col_character(),
  `2017` = col_character(),
  `2016` = col_character(),
  `2015` = col_character(),
  `2014` = col_character()
)

Warning message:
“NAs introduced by coercion”


In [5]:
us_pop <- read_csv('/pool001/mfzhao/geo_data/cc-est2018-alldata.csv')

us_pop %>%
    unite('key', STATE, COUNTY, sep = '') %>%
    filter(YEAR == 11, 
           AGEGRP==0) %>% 
    group_by(key) %>%
    summarize(n = sum(TOT_POP)) -> us_pop

Parsed with column specification:
cols(
  .default = col_double(),
  SUMLEV = col_character(),
  STATE = col_character(),
  COUNTY = col_character(),
  STNAME = col_character(),
  CTYNAME = col_character()
)

See spec(...) for full column specifications.



In [6]:
eu_mobility %>% 
    inner_join(eu_pop) -> eu_mobility

us_mobility %>% 
    inner_join(us_pop) -> us_mobility

Joining, by = "key"

Joining, by = "key"



In [7]:
write_csv(eu_mobility, '/pool001/mfzhao/PROCESSED_DATA/eu_mobility.csv')
write_csv(us_mobility, '/pool001/mfzhao/PROCESSED_DATA/us_mobility.csv')